#### TRANSFORM `PAYMENTS` DATA
- Map payment_status to contain descriptive values
(1-Success, 2-Pending, 3-Cancelled, 4-Failed)
- Write transformed data to Silver schema 


In [0]:
%sql
SELECT * FROM gizmobox_dev.bronze.payments_external LIMIT 10

#### CREATE FUNCTION TO EXTRACT `PAYMENT_STATUS`

In [0]:
%sql
CREATE FUNCTION IF NOT EXISTS gizmobox_dev.default.get_payment_status(payment_status INT)
RETURNS STRING
RETURN CASE 
  WHEN payment_status = 1 THEN 'SUCCESS'
  WHEN payment_status = 2 THEN 'PENDING'
  WHEN payment_status = 3 THEN 'CANCELLED'
  WHEN payment_status = 4 THEN 'FAILED'
  ELSE 'UNKNOWN' END

In [0]:
%sql
DESC FUNCTION EXTENDED gizmobox_dev.default.get_payment_status

In [0]:
%sql
SELECT
payment_id,
customer_id,
CAST(date_format(payment_date, 'yyyy-MM-dd') AS DATE) AS payment_date,
date_format(payment_date, 'HH:mm:ss') AS payment_time,
payment_status,
CASE 
  WHEN payment_status = 1 THEN 'SUCCESS'
  WHEN payment_status = 2 THEN 'PENDING'
  WHEN payment_status = 3 THEN 'CANCELLED'
  WHEN payment_status = 4 THEN 'FAILED'
  ELSE 'UNKNOWN' END AS payment_status_description,
  gizmobox_dev.default.get_payment_status(payment_status) AS payment_status_details,
payment_method
FROM
gizmobox_dev.bronze.payments_external

#### LOAD TRANSFORMED DATA INTO SILVER SCHEMA AS DELTA TABLE

In [0]:
%sql
CREATE TABLE IF NOT EXISTS gizmobox_dev.silver.payments
AS
SELECT
payment_id,
customer_id,
date_format(payment_date, 'dd-MM-yyyy') AS payment_date,
date_format(payment_date, 'HH:mm:ss') AS payment_time,
payment_status,
CASE 
  WHEN payment_status = 1 THEN 'SUCCESS'
  WHEN payment_status = 2 THEN 'PENDING'
  WHEN payment_status = 3 THEN 'CANCELLED'
  WHEN payment_status = 4 THEN 'FAILED'
  ELSE 'UNKNOWN' END AS payment_status_description,
payment_method
FROM
gizmobox_dev.bronze.payments_external

In [0]:
%sql
SELECT * FROM gizmobox_dev.silver.payments LIMIT 10

In [0]:
dbutils.notebook.exit('PAYMENTS TABLE HAS BEEN LOADED IN SILVER SCHEMA....')